# DACON 공모전
## 0220 김영은
### pytorch로 faster r-cnn 구현

In [6]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install imutils

  Stored in directory: C:\Users\yeongeunkim\AppData\Local\pip\Cache\wheels\db\23\45\fc7424906880ffa9577a2a428b961f2b79e0e21d9f71e7e6bc
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image
from os import listdir
from os.path import join, isfile

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정

In [2]:
dirty_mnist_answer = pd.read_csv("C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_mnist_2nd_answer.csv")
# dirty_mnist라는 디렉터리 속에 들어있는 파일들의 이름을 
# namelist라는 변수에 저장
namelist = os.listdir("C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst")

In [3]:
namelist

['49715.png',
 '49716.png',
 '49717.png',
 '49718.png',
 '49719.png',
 '49720.png',
 '49721.png',
 '49722.png',
 '49723.png',
 '49724.png',
 '49725.png',
 '49726.png',
 '49727.png',
 '49728.png',
 '49729.png',
 '49730.png',
 '49731.png',
 '49732.png',
 '49733.png',
 '49734.png',
 '49735.png',
 '49736.png',
 '49737.png',
 '49738.png',
 '49739.png',
 '49740.png',
 '49741.png',
 '49742.png',
 '49743.png',
 '49744.png',
 '49745.png',
 '49746.png',
 '49747.png',
 '49748.png',
 '49749.png',
 '49750.png',
 '49751.png',
 '49752.png',
 '49753.png',
 '49754.png',
 '49755.png',
 '49756.png',
 '49757.png',
 '49758.png',
 '49759.png',
 '49760.png',
 '49761.png',
 '49762.png',
 '49763.png',
 '49764.png',
 '49765.png',
 '49766.png',
 '49767.png',
 '49768.png',
 '49769.png',
 '49770.png',
 '49771.png',
 '49772.png',
 '49773.png',
 '49774.png',
 '49775.png',
 '49776.png',
 '49777.png',
 '49778.png',
 '49779.png',
 '49780.png',
 '49781.png',
 '49782.png',
 '49783.png',
 '49784.png',
 '49785.png',
 '4978

In [4]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print(DEVICE, torch.cuda.get_device_name(0))
else:
    DEVICE = torch.device("cpu")
    print(DEVICE)

cpu


In [5]:
# numpy로 돌려야한다고 오류떠서 numpy 배열로 하기
# 모든 이미지를 하나의 numpy 배열로 결합
# images 정의하기
image = Image.open('C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst/49715.png')
#image
import glob
filelist = glob.glob('C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst/*.png')
filelist

['C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49715.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49716.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49717.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49718.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49719.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49720.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49721.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49722.png',
 'C:/Users/yeongeunkim/Desktop/github/boazhappy repository/DACON_제 2회 컴퓨터 비전 학습 경진대회/dirty_minst\\49723.png',
 'C:/Users

In [6]:
images = np.array([np.array(Image.open(fname)) for fname in filelist])
images

array([[[225,  56, 220, ..., 134, 241, 147],
        [139, 240,  65, ..., 146, 201,  29],
        [ 57,   5,  37, ..., 163, 122, 167],
        ...,
        [ 50, 215,  31, ...,  31, 106, 159],
        [144, 224, 175, ..., 190, 244, 203],
        [182,  90, 121, ..., 141,  23,  92]],

       [[116, 214, 198, ..., 126,  96,  67],
        [231, 186, 125, ..., 214, 193, 249],
        [221, 125, 122, ..., 142, 128, 116],
        ...,
        [220,  61, 110, ..., 254, 151, 114],
        [191, 253, 160, ..., 146,  45, 248],
        [218, 140, 199, ..., 173,  12, 154]],

       [[ 13, 107, 176, ..., 139, 133,  19],
        [114,  72,  46, ...,   3,  70, 174],
        [250,  11, 183, ...,  48,  36, 158],
        ...,
        [189, 143, 150, ..., 246, 197, 102],
        [165, 175, 220, ..., 139, 232, 193],
        [198, 204,  94, ...,  66, 173, 172]],

       ...,

       [[240, 173, 141, ..., 136,  51, 102],
        [187, 160,  66, ..., 199,  98,  65],
        [ 86, 240, 159, ..., 202, 121, 217

In [12]:
images.shape

(285, 256, 256)

In [23]:
model = torchvision.models.vgg16(pretrained=True).to(DEVICE)
features = list(model.features)
print(len(features))
print(features)

31
[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace), Conv2d(512, 512, k